# Analysis of Multi-Omics Data of CLL Patients

## Overview

The data consist of $N=200$ blood samples from a cohort of Chronic Lymphocytic Leukemia (CLL) patients, where four omics data types were profiled: DNA methylation (450K Illumina microarray), bulk RNA-seq, somatic mutations and ex-vivo drug response assay. The dataset was introduced in detail by @Dietrich:JCI:2018 and can be downloaded [here](http://bioconductor.org/packages/release/data/experiment/html/BloodCancerMultiOmics2017.html). 
Its MOFA analysis was originally published by @Argelaguet:MolSysBiol:2018, and refined by @Lu:NatCancer:2021.

A detailed explanation of the data can be found [in this paper](https://www.jci.org/articles/view/93801). 

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
import scanpy as sc
import muon as mu
import pandas as pd
import numpy as np
import seaborn as sns
import mofax as mfx
import matplotlib.pyplot as plt

## Load Data

We load the data and ensure we have features in the columns, and samples in the rows:

In [ ]:
mrna = pd.read_csv("data/cll_mrna.csv", index_col=0).T
drugs = pd.read_csv("data/cll_drugs.csv", index_col=0).T
mutations = pd.read_csv("data/cll_mutations.csv", index_col=0).T
methylation = pd.read_csv("data/cll_methylation.csv", index_col=0).T

drug_names = pd.read_csv("data/drugs.txt", sep=",", index_col=0)
metadata = pd.read_csv("data/metadata.txt", sep="\t", index_col=0)

print(f"RNA Shape: {mrna.shape}")
print(f"Drugs Shape: {drugs.shape}")
print(f"Mutations Shape: {mutations.shape}")
print(f"Methylation Shape: {methylation.shape}")
print(f"Metadata Shape: {metadata.shape}")
print(f"Drug Names Shape: {drug_names.shape}")

## Data Properties

### mRNA expression

The mRNA expression were normalised by library size, followed by a variance stabilizing transformation using [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) in R: Statistical methods like clustering and PCA, commonly used in exploratory analysis of multidimensional data such as gene expression levels in RNA-Seq data, assume homoskedasticity, where the variability of data points is constant across different mean values. However, RNA-Seq data often exhibit heteroskedasticity, meaning the variance of gene expression levels increases with their mean. This discrepancy necessitates the use of specialized statistical approaches that can accurately account for the variable spread of data points to ensure reliable analysis or require preprocessing the data.

In [ ]:
mrna.head()

In [ ]:
sns.histplot(mrna.values.flatten(), bins=50, kde=False)

### DNA Methylation

DNA methylation was calculated for every CpG site using the [M-value](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-11-587), which provides a better summary statistic for downstream analysis. For the MOFA analysis we selected the top 1% ($N=4248$) most variable sites.

In [ ]:
methylation.head()

In [ ]:
sns.histplot(methylation.values.flatten(), bins=50, kde=False)

### Somatic Mutations
Mutations were assessed using a panel of common cancer mutations and are summarised in a binary format (0=no mutation, 1=mutaton):


In [ ]:
mutations.head()

In [ ]:
sns.histplot(mutations.values.flatten(), bins=50, kde=False)

### Drugs

The authors measured the effect of multiple drugs *ex vivo* using a high-throughput platform. For each drug they used 5 concentrations. The value reported is the viability score (0=all cells died, 1=no cells died). 

The mapping of Drug ID to Compound and Main Target can be found in App. 6: https://www.jci.org/articles/view/93801/sd/1

In [ ]:
drugs.head()

In [ ]:
sns.histplot(drugs.values.flatten(), bins=50, kde=False)

### <span style="color:red">Questions</span>

1) Compute and visualize (e.g., using a heatmap) the number of missing samples and features per modality  
  A: ...  
2) What are common strategies to impute these values?  
  A: ...  

### Metadata

Important columns are:  

- **Gender**: m (male), f (female)
- **Age**: age in years
- **TTT**: time (in years) which passed from taking the sample to the next treatment
- **TTD**: time (in years) which passed from taking the sample to patients' death
- **treatedAfter**: (TRUE/FALSE)
- **died**: whether the patient died (TRUE/FALSE)

In [ ]:
metadata.head()

For some of the correlation analysis in the following, we want to treat `sex` (*Gender* is a sociological concept, *sex* a biological. While they are related, in analyses such as here we usually care more about the latter) and `died` as numeric variables, and hence we create these here. 

In [ ]:
metadata["sex"] = metadata["Gender"].map({"m": 0, "f": 1})
metadata["died"] = metadata["died"].map({True: 1.0, False: 0.0})

#### Rename mRNA from ENSEMBL IDs
E.g., ENSG00000223972 to gene names DDX11L1

In [ ]:
gene_ids = pd.read_csv("data/cll_gene_ids.csv", index_col=0)
cols = list(mrna.columns)

# Replace each value in cols with the corrsponding value in the gene_ids dataframe
cols = [gene_ids.loc[gene_ids["GENEID"] == gene, "SYMBOL"].item() for gene in cols]
mrna.columns = cols

# avoid duplicated names with the Mutations view
mutations.columns = [f"m_{x}" for x in mutations.columns]

#### Rename drug names
E.g., D_001 to drug name navitoclax

### <span style="color:red">Questions</span>

1) Replace the drug names

## Data Preprocessing for Analysis

AnnData documentation: https://anndata.readthedocs.io/en/latest/

In [ ]:
mrna = sc.AnnData(mrna)
methylation = sc.AnnData(methylation)
drugs = sc.AnnData(drugs)
mutations = sc.AnnData(mutations)

### mRNA

To improve the computational efficiency, we focus our analysis on the top `k` highly variable genes. Feel free, to vary this number based on your hardware.

In [ ]:
# Select highly variable genes
sc.pp.highly_variable_genes(mrna, n_top_genes=2500)
mrna = mrna[:, mrna.var.highly_variable]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12.5, 5))
sns.heatmap(mrna.X)
ax.set_xlabel("Genes")
ax.set_ylabel("Cells")
plt.show()

### Create MUON object

Muon is built to handle MuData (multimodal data) objects, similar to how scanpy and AnnData focus on scRNA-seq data in Python, with individual modalities within MuData being represented using AnnData objects.

It's a convenient way to store, manage and process data from multiple modalities. See the package documentation [here](https://muon.readthedocs.io/en/latest/).

In [ ]:
mods = {
    "mrna": mrna,
    "methylation": methylation,
    "drugs": drugs,
    "mutations": mutations,
}

mdata = mu.MuData(mods)
mdata.obs = mdata.obs.join(metadata)
mdata

Muon allows you to run a variety of analyses on your data, e.g., MOFA with `muon.pl.mofa`, directly.

## Train a MOFA model

Now, we are going to create a MOFA object and then an integration optimization.

Some selected arguments of the MOFA wrapper are:
- *n_factors*: number of factors
- *likelihoods*: likelihood per view (options are “gaussian”, “poisson”, “bernoulli”). Default is guessed from the data.
- *ard_weights*: use ARD prior on the weights? Default is TRUE if using multiple views.
- *ard_factors*: use ARD prior on the factors? Default is TRUE if using multiple groups.
- *spikeslab_weights*: use spike-slab sparsity prior in the weights? default is TRUE.
- *spikeslab_factors*: use spike-slab sparsity prior in the factors? default is FALSE.
- *convergence_mode*: fast, medium, or slow convergence 
- *use_obs*: optional: strategy to deal with samples (cells) not being the same across modalities ("union" or "intersection")

### <span style="color:red">Questions</span>

1) What about missing data points?  
    A: ...
2) What likelihoods should we pass to the model? Is it a problem, if we chose an incorrect one?  
    A: ...
3) How can we find a good number of factors?  
    A: ...

In [ ]:
mu.tl.mofa(
    mdata,
    use_obs="union",
    n_factors=10,
    convergence_mode="medium",
    outfile="models/cll_v2.hdf5",
    save_metadata=True,
    save_data=True,
    verbose=False,
)

## Analysis and Interpretation of the Results

Scatter plot in MOFA factors coordinates

In [ ]:
mdata.obs.IGHV = (
    mdata.obs.IGHV.astype(str)
    .astype("category")
    .cat.rename_categories({"M": "mutated", "U": "unmutated"})
)
mu.pl.mofa(mdata, color="IGHV")

In [ ]:
# Load the model with mofax
model = mfx.mofa_model("models/cll_v2.hdf5")
model

##### Weights & Factors

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12.5, 5))
sns.heatmap(model.get_factors(), ax=ax[0], cmap="seismic", center=0)
sns.heatmap(model.get_weights(), ax=ax[1], cmap="seismic", center=0)
ax[0].set_title("Factors")
ax[1].set_title("Weights")

##### Factor Correlation

A good sanity check is to verify that the Factors are largely uncorrelated. In MOFA there are no orthogonality constraints such as
in Principal Component Analysis, but if there is a lot of correlation between Factors this suggests a poor model fit. Reasons?
Perhaps you used too many factors or perhaps the normalisation is not adequate.

In [ ]:
mfx.plot_weights_correlation(model)

##### Variance Explained

The most important insight that MOFA generates is the variance decomposition analysis. This plot shows the percentage of
variance explained by each factor across each data modality (and group, if provided). It summarises the source
of variation from a complex heterogeneous data set in a single figure.

In [ ]:
mfx.plot_r2(model, vmax=15, x="View")

What insights from the data can we learn just from inspecting this plot?

    Factor 1 captures a source of variability that is present across all data modalities. Thus, its etiology is likely to be something very important for the disease
    Factor 2 captures a very strong source of variation that is exclusive to the drug response data.
    Factor 4 captures variation that is present across multiple data modalities, except for DNA methylation. This is likely to be important too.
    Factor 3 is capturing some co-variation between the mRNA and the drug response assay.


### <span style="color:red">Questions</span>

Based on the MOFA output, if you were to profile just one molecular layer, which one would you choose to maximise the amount of sources of variation captured?

While trying to annotate factors, a global overview of top features defining them could be helpful.

In [ ]:
mfx.plot_weights_heatmap(
    model,
    # n_features=5,
    factors=range(0, 7),
    xticklabels_size=6,
    w_abs=True,
    cmap="viridis",
    cluster_factors=False,
)

In [ ]:
ax = mfx.plot_weights(
    model,
    views="mrna",
    factor=0,
    n_features=5,
    y_repel_coef=0.04,
    x_rank_offset=-150,
)

### Analysis: Factor 1 | "Give it a Name"

Ways to understande the underlying signal of a factor:  
    - association between the metadata and the factor values  
    - inspection of factor values  
    - inspection of feature weights  
    - GSEA of the mRNA weights  

Each factor captures a different source of variability in the data. Mathematically, each Factor is defined by a linear combination of the input features. Each Factor ordinates cells along a one-dimensional axis that is centered at zero. Samples with different signs manifest opposite phenotypes along the inferred axis of variation, with higher absolute value indicating a stronger effect.
Note that the interpretation of MOFA factors is analogous to the interpretation of the principal components in PCA.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 7.5))
sns.heatmap(model.get_weights(["mrna"]).T, center=0, cmap="vlag", ax=ax[0, 0])
sns.heatmap(model.get_weights(["methylation"]).T, center=0, cmap="vlag", ax=ax[0, 1])
sns.heatmap(model.get_weights(["mutations"]).T, center=0, cmap="vlag", ax=ax[1, 0])
sns.heatmap(model.get_weights(["drugs"]).T, center=0, cmap="vlag", ax=ax[1, 1])

#### Association analysis

Let's test for associations between the MOFA factors and some of the covariates:


In [ ]:
# Compute log10 of the p-values between factors and covariates
mfx.plot_factors_covariates_correlation(
    model, covariates=mdata.obs[["sex"]], pvalues=True
)

E.g. `sex` shows an associations with the survival outcome (whether the patients were deceased). 
We will explore association with clinical measurements later in the tutorial.  

#### Inspection of factor values


How do we interpret the factor values?
Each factor captures a different source of variability in the data. Mathematically, each Factor is defined by a linear combination of the input features. As the data is centered prior to running MOFA, each Factor ordinates cells along a one-dimensional axis that is centered at zero. Samples with different signs manifest opposite phenotypes along the inferred axis of variation, with higher absolute value indicating a stronger effect. Note that the interpretation of MOFA factors is analogous to the interpretation of the principal components in PCA.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
weights_fac_0 = model.get_factors(df=True).iloc[:, 0]
plt.scatter(range(200), weights_fac_0, c=weights_fac_0)
ax.axhline(0, color="k", linestyle="--")

#### Inspection of feature weights


By looking at the variance explained plot, we saw that Factor 1 captures variation in all data modalities. Out of all omics, the somatic mutation data is a good place to start, as somatic mutations are very sparse, easy to interpret and any change in the DNA is likely to have downstream consequences to all other molecular layers. Let’s plot the weights:

**How do we interpret the feature weights?**  
The weights provide a score for each feature on each factor. Features with no association with the corresponding factor are expected to have values close to zero, whereas features with strong association with the factor are expected to have large absolute values. The sign of the weights indicates the direction of the effect: a positive weights indicates that the feature has higher levels in the cells with positive factor values, and vice-versa. 

#### Plot feature weights for somatic mutations

By looking at the variance explained plot, we saw that Factor 1 captures variation in all data modalities. Out of all omics, the somatic mutation data is a good place to start, as somatic mutations are very sparse, easy to interpret and any change in the DNA is likely to have downstream consequences to all other molecular layers. Let's plot the weights:

In [ ]:
mfx.plot_weights(
    model,
    views=["mutations"],
    factors=0,
    zero_line=True,
    ncols=1,
    label_size=10,
    n_features=6,
)

Notice that most features lie at zero, indicating that most features have no association with Factor 1. There is however one gene that clearly stands out: IGHV (immunoglobulin heavy chain variable region). [This is the main clinical marker for CLL](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6355490/).  

IGHV has a positve weight. This means that samples with positive Factor 1 values have IGHV mutation whereas samples with negative Factor 1 values do not have the IGHV mutation. To confirm this, let’s plot the Factor values and colour the IGHV mutation status.

In [ ]:
df_plot = pd.merge(
    model.get_factors(df=True).iloc[:, 0],
    model.metadata.IGHV,
    left_index=True,
    right_index=True,
)

sns.violinplot(x="IGHV", y="Factor1", data=df_plot, hue="IGHV")
sns.swarmplot(x="IGHV", y="Factor1", data=df_plot, hue="IGHV", color="k", alpha=0.5)

#### Plot gene weights for mRNA expression


From the variance explained plot we know that Factor 1 drives variation across all data modalities. Let’s visualise the mRNA expression changes that are associated with Factor 1:

In [ ]:
mfx.plot_weights(
    model,
    views=["mrna"],
    factors=0,
    zero_line=True,
    ncols=1,
    label_size=10,
    n_features=6,
)

#### Plot molecular signatures in the input data 


In this case we have a large amount of genes that have large positive and negative weights. Genes with large positive values will be more expressed in the samples with IGHV mutation, whereas genes with large negative values will be more expressed in the samples without the IGHV mutation. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
weights_fac_1 = model.get_factors(df=True).iloc[:, 0]
df_plot = pd.DataFrame(
    mdata.mod["mrna"].X,
    columns=mdata.mod["mrna"].var_names,
    index=mdata.mod["mrna"].obs_names,
)
df_plot = df_plot[["ZNF667"]]
df_plot = pd.merge(
    df_plot, model.get_factors(df=True).iloc[:, 0], left_index=True, right_index=True
)
df_plot = pd.merge(df_plot, metadata[["IGHV"]], left_index=True, right_index=True)
df_plot["IGHV"] = df_plot["IGHV"].astype(str)

sns.scatterplot(y="ZNF667", x="Factor1", data=df_plot, hue="IGHV", s=40)

In [ ]:
# Plot distribution of LPL for different IGHV status
df_plot = pd.merge(
    model.get_factors(df=True).iloc[:, 0],
    model.metadata.IGHV,
    left_index=True,
    right_index=True,
)
# Join the expression of LPL gene for each sample
df_plot = pd.merge(
    df_plot,
    pd.DataFrame({"LPL": mrna[:, "LPL"].X.squeeze()}, index=mrna.obs.index),
    left_index=True,
    right_index=True,
)

sns.swarmplot(x="IGHV", y="Factor1", data=df_plot, hue="LPL")

In [ ]:
mfx.plot_weights(
    model,
    views=["drugs"],
    factors=0,
    zero_line=True,
    ncols=1,
    label_size=10,
    n_features=6,
)

Focusing on the drug response, we notice PF477736 (D_078), AZD7762 (D_020), AT13387 (D_017), and dasatinib (D_050) have the major association with this factor. Those are the exact top associations with the IGHV status described in the original paper:

"Indeed, the strongest associations of response with IGHV status were observed for dasatinib
and for 3 of the drugs already discussed above, the HSP90 inhibitor
AT13387 and the CHEK inhibitors PF477736 and AZD7762. These
results show how the critical role of BCR signaling renders CLL
cells sensitive to a broad range of kinase inhibitors that act by mul-
tiple target engagement of BCR components." from https://www.huber.embl.de/pub/pdf/Dietrich2018.pdf 

### GSEA

In [ ]:
import gseapy as gp
from gseapy import barplot, dotplot

In [ ]:
fac1 = model.get_weights(["mrna"])[:, 3]
# Get gene names of all genes with a weight > 0.6
gene_list = list(mrna.var_names[fac1 > 0.5])

fig, ax = plt.subplots(1, 1, figsize=(10, 2))
sns.histplot(fac1, ax=ax)
ax.set_title(f"Factor | {len(gene_list)}")
plt.show()

In [ ]:
# if you are only intrested in dataframe that enrichr returned, please set outdir=None
enr = gp.enrichr(
    gene_list=gene_list,
    gene_sets=["MSigDB_Hallmark_2020", "KEGG_2021_Human", "Reactome_2013"],
    organism="human",
    outdir=None,
    cutoff=0.05,
)

In [ ]:
# Sort by adjusted p-value
enr.results.sort_values("Adjusted P-value").head(8)

In [ ]:
ax = barplot(
    enr.results,
    column="Adjusted P-value",
    group="Gene_set",  # set group, so you could do a multi-sample/library comparsion
    size=10,
    top_term=5,
    figsize=(3, 5),
    color={
        "MSigDB_Hallmark_2020": "red",
        "KEGG_2021_Human": "blue",
        "Reactome_2013": "green",
    },
)

### End

In [ ]:
# it's always a good idea to close the model after using it
model.close()